In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
from datetime import datetime, timedelta
import asyncio
from concurrent.futures import ThreadPoolExecutor
import nest_asyncio
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
def get_previous_day(formatted_date):
    # Parse the formatted date
    date_obj = datetime.strptime(formatted_date, "%Y-%m-%d")
    
    # Calculate the previous date
    previous_date = date_obj - timedelta(days=1)
    
    # Return only the day of the previous date
    previous_day = previous_date.day
    return previous_day

In [16]:
# Initialize WebDriver
def accept_cookies(driver):
    """Accepts cookies when prompted, used after a new driver is open."""
    time.sleep(10)
    try:
        accept_button = driver.find_element(By.ID, "onetrust-accept-btn-handler")
        accept_button.click()
        time.sleep(1)  # Pause to allow the click to register
    except Exception as e:
        print(f"Cookie acceptance button not found: {e}")

In [17]:
def to_csv(res, year):
    """Sorts and writes the current data to csv."""
    df = pd.DataFrame(res, columns=['year', 'team', 'games', 'date', 'wins', 'losses', 'WCGB'])

    # Sort the DataFrame
    df_sorted = df.sort_values(by=['year', 'team', 'games'])
        
    path = f"L:/RA_work/JAY/datasets/final/non_finalcopies/{year}.csv"
    df_sorted.to_csv(path, index=False)

In [18]:
def read_html(year, date, driver):
    """
    Extracts data from a main table.

    Args:
        year (int): The year for which the data is being extracted.
        date (str): The date for which the data is being extracted.
        driver (WebDriver): The Selenium WebDriver instance.

    Returns:
        list: A list of extracted data rows, where each row contains information about a team, game stats, and the specified date.
    """
    total = []
    try:
        
        main_table = driver.find_element(By.CLASS_NAME, "tablestyle__StyledTable-sc-wsl6eq-0")
        
        
        if not main_table:
            raise ValueError("Main table with class 'tablestyle__StyledTable-sc-wsl6eq-0' not found.")
        
        # Find all tbody elements within the main table
        tbodies = main_table.find_elements(By.TAG_NAME, "tbody")
        
        # These indices are which tbody elements contain info (the others are headers)
        indices_to_extract = [0, 2, 4, 6, 8, 10]
        
        for index in indices_to_extract:
            
            if index < len(tbodies):
                tbody = tbodies[index]
                
                rows = tbody.find_elements(By.TAG_NAME, "tr")
                
                for row in rows:
                    # Get team name from th
                    th = row.find_element(By.TAG_NAME, "th")
                    team_name = th.find_element(By.TAG_NAME, "a").text
                    
                    # Get remaining row data
                    cells = row.find_elements(By.TAG_NAME, "td")
                    cell_text = [cell.text for cell in cells]
                    game = int(cell_text[0]) + int(cell_text[1]) + 1
                    output = [year, team_name, game, date, cell_text[0], cell_text[1], cell_text[4]]
                    print(output)
                    # Print the team name and row data for debugging purposes
                    total.append(output)
            else:
                print(f"Tbody index {index} is out of range.")
    except Exception as e:
        print(f"Exception in read_html: {str(e)}")
    finally:
        return total



def run(year, driver):
    """
    Loads and parses through mlb site to get data for a particular year. (Each year has a separate URL)

    Args:
        year (int): The year for which the data is being retrieved.
        driver (WebDriver): The Selenium WebDriver instance.

    Returns:
        list: A list of all extracted data rows across all selected dates in the year.
    """
    res = []
    driver.get(f"https://www.mlb.com/standings/{year}")
    accept_cookies(driver)
    try:
        wait = WebDriverWait(driver, 10)
        
        def select_menu():
            """Opens and selects the calendar menu."""
            date_picker_input = wait.until(EC.visibility_of_element_located((By.ID, "pickerInput")))
    
            driver.execute_script("arguments[0].scrollIntoView(true);", date_picker_input)
    
            try:
                date_picker_input.click()
            except Exception as e:
                driver.execute_script("arguments[0].click();", date_picker_input)

      
        def tail():
            """Interacts with the button used to select date after the calendar is open, then writes current data to csv."""
            date_label = driver.find_element(By.CLASS_NAME, "datePickerstyle__DateLabel-sc-1ewrv5d-8")
            current_date = date_label.text
            _, month_name, day = current_date.split()
            month_number = datetime.strptime(month_name, "%b").month
            formatted_date = f"{year}-{month_number:02d}-{int(day):02d}"
            print(f"Current date: {formatted_date}")
            
            res.extend(read_html(year, formatted_date, driver)) # Getting the stats
            to_csv(res, year)
            # Get the currently selected day
            selected_day_element = driver.find_element(By.CLASS_NAME, "react-datepicker__day--selected")
            selected_day = selected_day_element.text
            day_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "react-datepicker__day")))
            
            # Attempt to find and click the previous day
            previous_day_element = None
            day_text = None
            
            for i, j in enumerate(day_elements):
                # If the first days that appear are disabled, do not set previous_day_element
                if j.get_attribute("aria-disabled") == "true":
                    continue
                elif j.get_attribute("aria-selected") == "true":
                    # If we reach the selected element stop the loop and keep previous element
                    # Must check if the selected element is the first to appear 
                    if i == 0:
                        prev_month_button = driver.find_element(By.XPATH, "//button[@aria-label='Previous Month']")
                        if prev_month_button.is_enabled():
                            prev_month_button.click()
                            time.sleep(0.2)
                            day_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "react-datepicker__day")))
                            last_day = day_elements[-1]
                            last_day.click()
                            return get_previous_day(formatted_date)
                        else:
                            print("No more previous months available.")  
                            break
                    else:
                        break
                
                previous_day_element = j
                day_text = j.text
        
            if previous_day_element:
                previous_day_element.click()
                time.sleep(0.2)
                return day_text
            else:
                print("No selectable previous day found")
                return None
        
        
            
            
        def select_menu():
            date_picker_input = wait.until(EC.visibility_of_element_located((By.ID, "pickerInput")))
            driver.execute_script("arguments[0].scrollIntoView(true);", date_picker_input)
            try:
                date_picker_input.click()
            except Exception as e:
                driver.execute_script("arguments[0].click();", date_picker_input)

        select_menu()
        last_day = tail()
        stop = 0
        while last_day: 
            time.sleep(6)
            select_menu()
            last_day = tail()
            stop += 1

    except Exception as e:
        print(f"Exception: {str(e)}")

    finally:
        driver.quit()
        
        
        return res

In [ ]:
# Run this to start
# Edit buffer size to determine how many years will be scraped at once
nest_asyncio.apply()
async def process_batch(batch):
    with ThreadPoolExecutor(max_workers=len(batch)) as executor:
        loop = asyncio.get_event_loop()
        
        tasks = [loop.run_in_executor(executor, run, year, webdriver.Chrome()) for year, _ in batch]
        await asyncio.gather(*tasks)

async def main():
    temp = [(x, f"https://www.mlb.com/standings/wild-card/{x}") for x in range(2011, 2014)]
    batch_size = 3

    for i in range(0, len(temp), batch_size):
        batch = temp[i:i + batch_size]
        await process_batch(batch)

# Run the async main function
await main()

In [11]:
def process_game_data(files):
    # Merge and sort data
    dataframes = [pd.read_csv(file) for file in files]
    merged_df = pd.concat(dataframes)
    merged_df = merged_df.sort_values(by=['year', 'team', 'games']).reset_index(drop=True)
    
    # Remove duplicates
    merged_df = merged_df.drop_duplicates(subset=['year', 'team', 'games'])
    
    # Add buffer rows
    buffer_rows = []
    grouped = merged_df.groupby(['year', 'team'])
    
    for (year, team), group in grouped:
        min_game = group['games'].min()
        max_game = group['games'].max()
        existing_games = set(group['games'])
        
        # Find missing games and add buffer rows
        for game in range(min_game, max_game + 1):
            if game not in existing_games:
                buffer_rows.append([year, team, game, '', '', '', ''])
    
    
    buffer_df = pd.DataFrame(buffer_rows, columns=merged_df.columns)
    final_df = pd.concat([merged_df, buffer_df]).sort_values(by=['year', 'team', 'games']).reset_index(drop=True)
    
    return final_df


files = ["L:/RA_work/JAY/datasets/final/non_finalcopies/" + str(x) + ".csv" for x in range(2007, 2014)]
final_df = process_game_data(files)

final_df.to_csv("L:/RA_work/JAY/datasets/final/non_finalcopies/merged.csv", index=False)

In [12]:
def replace_plus():
    df = pd.read_csv("L:/RA_work/JAY/datasets/final/non_finalcopies/merged.csv")

    df['WCGB'] = df['WCGB'].astype(str).str.replace('+', 'up ', regex=False)
    
    df.to_csv("L:/RA_work/JAY/datasets/final/non_finalcopies/final2007to2013.csv", index=False)

In [13]:
def replace_dash():
    df = pd.read_csv("L:/RA_work/JAY/datasets/final/non_finalcopies/final2007to2013.csv")
    df['WCGB'] = df['WCGB'].astype(str).str.replace('+', 'up ', regex=False)
    df['WCGB'] = df['WCGB'].replace('nan', ' ')
    
    df.to_csv("L:/RA_work/JAY/datasets/final/non_finalcopies/playoff_odds_2007to2013.csv", index=False)